In [3]:
%pip install langchain-openai python-dotenv

  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached anyio-4.10.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.10.0-cp311-cp311-win_amd64.whl.metadata (5.3 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached regex-2025.7.34-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using ca


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
%pip install --upgrade --quiet  docx2txt langchain-community

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install -qU langchain-text-splitters

In [11]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [12]:
len(document_list)

225

In [13]:
from langchain_openai import OpenAIEmbeddings

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')# 3072

In [7]:
%pip install -qU langchain langchain-pinecone 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
%pip install -qU pinecone

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# pinecone 에 데이터 로딩
# import os
# from pinecone import Pinecone
# from langchain_pinecone import PineconeVectorStore

# index_name = 'tax-index2'
# pinecone_api_key = os.getenv('PINECONE_API_KEY')
# pc = Pinecone(api_key=pinecone_api_key)

# vectorstore = PineconeVectorStore.from_existing_index(
#     index_name=index_name,
#     embedding=embedding
# )

In [54]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index4'
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)
pinecone_index = pc.Index(index_name)

vectorstore = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)


# vectorstore = PineconeVectorStore(
#     index=pinecone_index,
#     embedding=embedding,
#     text_key='text'
# )

# texts = [doc.page_content for doc in document_list]
# metadatas = [doc.metadata for doc in document_list]

# batch_size = 2

# for i in range(0, len(texts), batch_size):
#     batch_texts = texts[i:i + batch_size]
#     batch_metadatas = metadatas[i:i + batch_size]

#     try:
#         vectorstore.add_texts(texts=batch_texts, metadatas=batch_metadatas)
#     except Exception as e:
#         print(f"Error adding batch {i // batch_size + 1}: {e}")

In [55]:
query = "연봉 5천만원인 거주자의 종합소득세는?"
#query = "연봉 5천만원 직장인의 소득세는?"

In [56]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
retriever.invoke(query)

[Document(id='dce9712c-7910-471d-ad69-35f1b7c958f9', metadata={'source': './tax.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서

In [57]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [ ]:
from langchain.prompts import PromptTemplate

# prompt = PromptTemplate(
#     input_variables=["context", "question"],
#     template="""
# [Identity]
# - 당신은 최고의 한국 소득세 전문가 입니다.
# - [Context]를 참고해서 사용자의 질문에 답변해 주세요.

# [Context]
# {context}

# Question: {question}
# """
# )

In [58]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

d:\Study\agetn_work3\myenv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
# from langchain.chains import RetrievalQA

# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     retriever=retriever,
#     chain_type_kwargs={"prompt": prompt},
# )

In [60]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
)

In [61]:
ai_message = qa_chain.invoke({"query": query})

In [ ]:
#ai_message = qa_chain({"query": query})

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20640\3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


In [62]:
ai_message

{'query': '연봉 5천만원인 거주자의 종합소득세는?',
 'result': '연봉 5천만 원인 거주자의 종합소득세는 84만 원에 1,400만 원을 초과하는 금액의 15퍼센트를 더한 금액입니다. 이를 계산하면, 3,600만 원의 15퍼센트는 540만 원으로, 최종 종합소득세는 624만 원이 됩니다.'}